# Push Docker Image to AWS Elastic Container Registry

In [ ]:
!cd container; ./build_and_push.sh drug-prediction-gnn

# Run Local Training Job

In [ ]:
# Download the dataset to local
! mkdir data

# download pretrained mol2vec model
!wget -O data/model_300dim.pkl https://github.com/samoturk/mol2vec/raw/master/examples/models/model_300dim.pkl model_300dim.pkl   

# download example data     
!wget -O data/ames.sdf https://github.com/samoturk/mol2vec/raw/master/examples/data/ames.sdf
    
#download hiv data
!wget -O data/hiv.zip https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/molnet_publish/hiv.zip
!cd data; unzip hiv.zip

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import os
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

In [ ]:
from sagemaker.estimator import Estimator

hyperparameters = {'dev-mode':False, 
                   'net-type': 'GCN',
                   'epochs': 2,
                   'learning-rate':1e-4,
                   'gcn-hidden-feats':128,
                   'classifier-hidden-feats':512,
                   'batch-size':200,
                   'num-hidden-layers':2}

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='drug-prediction-gnn:latest',
                      hyperparameters=hyperparameters)

estimator.fit('file:///home/ec2-user/SageMaker/graph_neural_net/data')

# Run A Sagemaker Training Job

In [ ]:
import sagemaker as sage

sess = sage.Session()

In [ ]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'drug-prediction-gnn'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

In [ ]:
from sagemaker.estimator import Estimator

data_location = 's3://jdurago-insight-2020a/data' # verify data has been uploaded to S3 bucket specified here
output_path = 's3://jdurago-insight-2020a/output/baseline/' # output path where models will be saved
max_run_time = 3*60*60 # train for max of 3 hours

hyperparameters = {'dev-mode':False, 
                   'net-type': 'GCN',
                   'epochs': 200,
                   'learning-rate':1e-4,
                   'gcn-hidden-feats':128,
                   'classifier-hidden-feats':512,
                   'batch-size':200,
                   'num-hidden-layers':6}

base_job_name = hyperparameters['net-type']

metric_definitions = [{"Name": "train:loss", 
                       "Regex": "train_loss:(\d+.\d+)"}, 
                      {"Name": "test:auc_roc", 
                       "Regex": "test_score:(\d+.\d+)"},
                      {"Name": "val:auc_roc", 
                       "Regex": "auc_roc:(\d+.\d+)"},
                      {"Name": "val:f1_score", 
                       "Regex": "f1_score:(\d+.\d+)"}
                     ]

instance_type = 'ml.m5.12xlarge'

tags = [{'Key': 'net-type',
         'Value': hyperparameters['net-type']}]

estimator = Estimator(role=role,
                      base_job_name = base_job_name,
                      train_instance_count=1,
                       train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                     output_path = output_path,
                     train_max_run=max_run_time,
                     metric_definitions=metric_definitions,
                     tags=tags)

estimator.fit(data_location)


# Hyperparameter Tuning

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.estimator import Estimator
import sagemaker as sage

sess = sage.Session()

max_jobs = 64
max_parallel_jobs = 4

data_location = 's3://jdurago-insight-2020a/data' # verify data has been uploaded to S3 bucket specified here
output_path = 's3://jdurago-insight-2020a/output/baseline/' # output path where models will be saved
max_run_time = 6*60*60 # train for max of 6 hours
instance_type = 'ml.m5.4xlarge'


hyperparameters = {'dev-mode':False, 
                   'net-type': 'GAT',
                   'epochs': 200,
                   'batch-size':200,
                   'learning-rate':1e-4}

base_tuning_job_name = 'hpm-tuning-' + hyperparameters['net-type']

tags = [{'Key': 'net-type',
         'Value': hyperparameters['net-type']}]

# For GCN
# hyperparameter_ranges = {'gcn-hidden-feats': CategoricalParameter([32,64,128,256,512]),
#                          'classifier-hidden-feats': CategoricalParameter([32,64,128,256,512]),
#                          'num-hidden-layers': CategoricalParameter([2,3,4,5,6])}

# For GAT
hyperparameter_ranges = {'gat-hidden-feats': CategoricalParameter([32,64,128]),
                         'classifier-hidden-feats': CategoricalParameter([64,128,256,512]),
                         'num-head-layers': CategoricalParameter([2,3,4,5,6]),
                         'head-output-size': CategoricalParameter([2,3,4,5,6])}


objective_metric_name = 'val:f1_score'
objective_type = 'Maximize'
metric_definitions = [{"Name": "train:loss", 
                       "Regex": "train_loss:(\d+.\d+)"}, 
                      {"Name": "test:auc_roc", 
                       "Regex": "test_score:(\d+.\d+)"},
                      {"Name": "val:auc_roc", 
                       "Regex": "auc_roc:(\d+.\d+)"},
                      {"Name": "val:f1_score", 
                       "Regex": "f1_score:(\d+.\d+)"}
                     ]

estimator = Estimator(role=role,
                      train_instance_count=1,
                       train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                     output_path = output_path,
                     train_max_run=max_run_time,
                     metric_definitions=metric_definitions,
                     tags=tags)

tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=max_jobs,
                            base_tuning_job_name = base_tuning_job_name,
                            max_parallel_jobs=max_parallel_jobs,
                            objective_type=objective_type)

tuner.fit(data_location)